<a href="https://colab.research.google.com/github/bboInCoding17/Test3/blob/master/01_%EB%A6%AC%EB%B7%B0_%EC%98%88%EC%B8%A1_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
### 런타임 유행 GPU로 변경

In [4]:
!pip install konlpy tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 19.6 MB/s eta 0:00:00


In [5]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
# 해당 url 에서 학습 데이터 및 테스트 데이터 txt 파일로 다운
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f9d8aa148e0>)

In [8]:
train_data  = pd.read_table('ratings_train.txt')
test_data  = pd.read_table('ratings_test.txt')

In [9]:
print(train_data)

              id                                           document  label
0        9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1        3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2       10265843                                  너무재밓었다그래서보는것을추천한다      0
3        9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4        6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
...          ...                                                ...    ...
149995   6222902                                인간이 문제지.. 소는 뭔죄인가..      0
149996   8549745                                      평점이 너무 낮아서...      1
149997   9311800                    이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?      0
149998   2376369                        청춘 영화의 최고봉.방황과 우울했던 날들의 자화상      1
149999   9619869                           한국 영화 최초로 수간하는 내용이 담긴 영화      0

[150000 rows x 3 columns]


In [10]:
print('훈려용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰개수 출력

훈려용 리뷰 개수 : 150000


In [11]:
#document 열과 label 열의 중복을 제외한 값의 개수
train_data['document'].nunique(), train_data['label'].nunique()

(146182, 2)

In [12]:
# 긍정(1)과 부정(0) 개수 확인
print(train_data.groupby('label').size().reset_index(name='count'))

   label  count
0      0  75173
1      1  74827


In [13]:
# 학습 데이터 null인 경우가 있는지
print(train_data.isnull().values.any())

True


In [14]:
# null인 데이터 개수
print(train_data.isnull().sum())

id          0
document    5
label       0
dtype: int64


In [15]:
#dropna => 해당 데이터프레임에 빈 값 제거
# how => 어떠한 조건으로 제거할 것인가
# * any : 행들 중 아무 값이라도 null 인 경우
# * all : 모든 값이 null인 경우
train_data = train_data.dropna(how='any') #파라미터는 하나 이상의 NaN 값이 있는 행을 제거하도록 지정
print(train_data.isnull().values.any()) #Null 값이 존재하는지 확인



False


In [16]:
#한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣\s]","")

train_data[:5]

<ipython-input-16-738ba0938240>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣\s]","")


,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [17]:
# null 이 된 값 출력
train_data.loc[train_data.document.isnull()][:5]

,id,document,label


In [18]:
train_data = train_data.dropna(how='any') # 파라미터는 하나 이상의 NaN 값이 있는 행을 제거하도록 지정
print(len(train_data))

149995


In [19]:
#test_data 전처리 작업

# 1. document열에서 중복인 내용이 있다면 중복 제거
test_data.drop_duplicates(subset=['document'], inplace =True) 

# 2. 정규식으로 한글과 공백을 제외하고 모두 제거
test_data['document']=test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") 

# 3. 문자열의 시작 부분에서 하나 이상의 공백 문자를 찾아 제거 후 공백.
test_data['document'] = test_data['document'].str.replace('^ +', "") 

# 4. 공백은 NaN으로 변경
test_data['document'].replace('', np.nan, inplace =True) 

# 5. NaN 값 제거 _ NaN값이 하나라도 있으면 행 삭제
test_data = test_data.dropna(how='any') 

# len() _ 개수 확인
print('전처리 후 테스트용 샘플의 개수 :', len(test_data))

전처리 후 테스트용 샘플의 개수 : 48852


<ipython-input-19-3a2ca84c68c6>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document']=test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-19-3a2ca84c68c6>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('^ +', "")


In [20]:
stopwords=['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']
okt = Okt()
okt.morphs('와 이런것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 듯')


['와', '이런', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만드는', '게', '나을', '듯']

In [21]:
X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    X_train.append(stopwords_removed_sentence)

X_train_tmp = X_train[:]

100%|██████████| 149995/149995 [13:59<00:00, 178.61it/s]


In [ ]:
X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) #토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    X_test.append(stopwords_removed_sentence)


  4%|▍         | 2136/48852 [00:13<03:24, 227.98it/s]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train) # 문자데이터를 받아 해당 문자에 index를 부여한다.
print(tokenizer.word_index)

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) #단어 수
rare_cnt =0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운드
total_freq =0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
  total_freq = total_freq + value

  # 단어의 등장 빈도수가 threshold보다 작으면
  if(value < threshold) :
    rare_cnt = rare_cnt +1
    rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :', total_cnt)
print('단어 빈도가 %s번 이하인 희귀 단어의 수 :%s'%(threshold -1, rare_cnt))
print("단어 집합에서 희귀단어의 비율 :",(rare_cnt/total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율 :", (rare_freq/total_freq)*100)

In [ ]:
# 전체 단어 개수 중 빈도수 2 이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 +1
vocab_size = total_cnt - rare_cnt +1
print('단어 집합의 크기 :', vocab_size)

tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train) # text를 정수형, 실수형으로 전환해서 작업하겠다.
X_test = tokenizer.texts_to_sequences(X_test) # text를 정수형, 실수형으로 전환해서 작업하겠다.

In [ ]:
y_train = np.array(train_data['label']) # x를 넣어서 y 값을 도출하기 위함
y_test = np.array(test_data['label'])

In [ ]:
y_train # 0이면 긍정/ 1이면 부정 => 원인에 대한 결과를 도출할 수 있다.

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) <1]
drop_train

In [ ]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

In [ ]:
print('리뷰의 최대 길이 :', max(len(review) for review in X_train))
print('리뷰의 최대 길이 :', sum(map(len, X_train))/len(X_train))

plt.hist([len(review) for review in X_train], bins = 50)
plt.xlabel('length of samples')
plt.ylabel('length of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
max_len=30
below_threshold_len(max_len, X_train)

In [ ]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
embedding_dim = 100
hidden_units = 128

model = Sequential() # 여러 모델을 합쳐서 작업할 수 있도록 해준다.(만들어진 모델 구성을 통해)
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose =1, patience=4) # 학습을 여러번 계속 반복 / 정확도가 오르다가 오르지 않는 순간을 catch 후 멈춘다)

mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc']) # 최적화 하기 위함

history = model.fit(X_train, y_train, epochs=15, callbacks=[es,mc],batch_size=64, validation_split=0.2) # fit 함수를 통해 학습이 들어간다. epochs _ 학습 횟수
# bath_size _ 한번 학습할 때 몇 개의 사이즈로 작업할 것인가/ 

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도 : %.4f" %\
      (loaded_model.evaluate(X_test, y_test)[1]))


In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True)
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측

  if(score > 0.5):
    print("{:2.f}% 확률로 긍정 리뷰입니다. ".format(score*100))
  else:
    print("{:2.f}% 확률로 부정 리뷰입니다. ".format(1-score)*100)

In [ ]:
sentiment_predict("영화가 재미있어요.")

In [ ]:
while True:
  pass